In [ ]:
!pip install yfinance
!pip install yahooquery
!pip install tvdatafeed
!pip install tensorflow
from tvDatafeed import TvDatafeed, Interval
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model
import os
import shutil
from datetime import datetime
import glob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def work_with_dir():
  if os.path.exists("/content/data/"):
    shutil.rmtree("/content/data/", ignore_errors=True)
    print("Data Folder Removed")
    os.mkdir("/content/data/")
  if not os.path.exists("/content/data/"):
    os.mkdir("/content/data/")
  if not os.path.exists("/content/extracted/"):
    os.mkdir("/content/extracted/")
def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["suggestion"], axis=1)
  y = data[["suggestion"]]
  y = pd.get_dummies(y)
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.2,random_state=99)
  print(xTrain.shape, end=" ")
  print(yTrain.shape)
  print(xTest.shape, end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def download_data(symbols,periodd,intervall):
  indexx = 100
  work_with_dir()
  for symbol in symbols:
    if ((symbols.index(symbol)+1) % 100 == 0):
      print(f" -- {indexx}",end="")
      indexx = indexx + 100
    try:
        data = yf.download(symbol, period=periodd,interval=intervall, progress=False,show_errors=False)
        if data.empty:
           pass
        else:
             data.to_csv(f"/content/data/{symbol}.csv")         
    except:
       print("Error!")
  print(" ")
def each_file_proc(files,now,index,min_range,max_range,how_many_future_candles,how_many_past_candles,each_row_past,interval_for_that):
     data = []
     unattached_dfs = []
     files = list(files)
     for file in files:
        #print(f"{files.index(file)+1+index}",end=" ")
        if (files.index(file)+index+1) % 10 == 0:
           print(f"{files.index(file)+1+index}",end=" ")
           if (files.index(file)+index+1) % 400 == 0:
             print(" ")

        address = f"/content/data/{file}"
        try:
           unattached_dfs.append(process(pd.read_csv(address),min_range,max_range,how_many_future_candles,how_many_past_candles,each_row_past,interval_for_that))
        except:
          pass
     nud = []
     for i in unattached_dfs:
       if i.size != 0:
         nud.append(i)
     data = np.concatenate(nud)
     data = pd.DataFrame(data, columns=clmns)
     right_now = datetime.now().strftime("%H%M%S%f")
     data.to_csv(f"/content/extracted/{now}/{right_now}.csv")  
def extract_data(pieces,min_range,max_range,how_many_future_candles,how_many_past_candles,each_row_past,interval_for_that):
  print(f"Files In Data : {len(os.listdir('/content/data/'))}")
  pd.options.mode.chained_assignment = None
  files = os.listdir("/content/data/")
  new_files = np.array_split(files, pieces)
  print("Processing File:")
  now = datetime.now().strftime("%H%M%S")
  os.mkdir(f"/content/extracted/{now}/")
  index = 0 
  for files in new_files:
     each_file_proc(files,now,index,min_range,max_range,how_many_future_candles,how_many_past_candles,each_row_past,interval_for_that)
     index = index + len(files)
  print(" ")
  return now
def delete_all_csv(now):
   path = f'/content/extracted/{now}/*.csv'
   files = glob.glob(path)
   for file in files:
       os.remove(file)
def make_df(now):
   path = f'/content/extracted/{now}/*.parquet'
   files = glob.glob(path)
   #data = pd.DataFrame()
   data = pd.DataFrame()
   for adr in files:
     data =pd.concat([data,pd.read_parquet(adr)])
   if "Unnamed: 0" in data:
     data.drop("Unnamed: 0",axis=1,inplace=True)
   print(data.shape)
   xTrain,xTest,yTrain,yTest = spliting(data)
   data.to_parquet(f'/content/extracted/{now}/data.parquet')
   delete_all_csv(now)
   data = []
   return xTrain,xTest,yTrain,yTest
def to_par(now,howmanyfiles): 
    files = os.listdir(f"/content/extracted/{now}/")
    addresses = []
    for file in files:
      addresses.append(f"/content/extracted/{now}/{file}")
    new_adr = np.array_split(addresses,howmanyfiles)
    for adrs in new_adr:
      datas = []
      for adr in adrs:
        datas.append(pd.read_csv(adr))
      rnow = datetime.now().strftime("%H%M%S%f")
      datas = pd.concat(datas)
      datas.to_parquet(f"/content/extracted/{now}/part_{rnow}.parquet")   
def scaler(row,min_range,max_range):
    scaler = MinMaxScaler(feature_range=(min_range, max_range))
    row = scaler.fit_transform(row)
    return row
def make_y(data, i,how_many_future_candles,how_many_past_candles):
  mean_of_next = []
  for ix in range(i,i+how_many_future_candles):
    mean_of_next.append((data[ix][3]-data[ix][0]))
  mean_of_next = sum(mean_of_next)
  mean_of_previous = []
  for ix in range(i-how_many_past_candles,i):
    mean_of_previous.append((data[ix][3]-data[ix][0]))
  mean_of_previous = sum(mean_of_previous)
  if mean_of_next > mean_of_previous:
    return "buy"
  else:
    return "sell"
def process(data,min_range,max_range,how_many_future_candles,how_many_past_candles,each_row_past,interval_for_that):
    data = data.dropna()
    row = []
    if len(data.columns) == 7:
      data = data.iloc[:, 1:]
    data = np.array(data)
    for i in range(each_row_past, data.shape[0]-10):
        opens = []
        highs = []
        lows = []
        closes = []           
        for ix in range(each_row_past, 1,-1):
            opens.append([data[i-ix][0] - data[i-ix+1][0] ])
            highs.append([data[i-ix][1] - data[i-ix+1][1] ])
            lows.append([data[i-ix][2] - data[i-ix+1][2] ])
            closes.append([data[i-ix][3] - data[i-ix+1][3] ])
        
        opens = scaler(np.array(opens).reshape(-1, 1),min_range,max_range)
        highs = scaler(np.array(highs).reshape(-1, 1),min_range,max_range)
        lows = scaler(np.array(lows).reshape(-1, 1),min_range,max_range)
        closes = scaler(np.array(closes).reshape(-1, 1),min_range,max_range)
        
        sugg = make_y(data, i,how_many_future_candles,how_many_past_candles)
        arr = np.concatenate((np.array(opens).flatten(), np.array(highs).flatten()
        , np.array(lows).flatten(), np.array(closes).flatten()), axis=0).reshape(-1, 1)
        #arr = scaler(arr.reshape(-1, 1),min_range,max_range)
        arr = np.append(arr, sugg)
        row.append(arr)
    return np.array(row)
def start(min_range,max_range,how_many_future_candles,how_many_past_candles,each_row_past,interval_for_that,into_how_many_csv,csv_to_parquet):
     folder_name = extract_data(into_how_many_csv,min_range,max_range,how_many_future_candles,how_many_past_candles,each_row_past,interval_for_that)
     to_par(folder_name,csv_to_parquet)
     xTrain,xTest,yTrain,yTest = make_df(folder_name)
     return xTrain,xTest,yTrain,yTest  

In [ ]:
symbols = get_crypto_syms()
print(f"Symbols : {len(symbols)}")
download_data(symbols,"max","1wk")

Symbols : 1500
Data Folder Removed
 -- 100 -- 200 -- 300 -- 400 -- 500 -- 600 -- 700 -- 800 -- 900 -- 1000 -- 1100 -- 1200 -- 1300 -- 1400 -- 1500 


In [ ]:
clmns = list(range(1, 117))
clmns.append("suggestion")

min_range,max_range = -1,1
how_many_future_candles = 5
how_many_past_candles = 5
each_row_past,interval_for_that = 30,-1

into_how_many_csv = 100
csv_to_parquet = 1

xTrain,xTest,yTrain,yTest = start(min_range,max_range,how_many_future_candles,how_many_past_candles,each_row_past,interval_for_that,into_how_many_csv,csv_to_parquet)
print(yTrain.value_counts())
print(yTest.value_counts())

In [ ]:
xTrain

In [ ]:
yTrain

In [ ]:
!pip install stable-baselines3[extra]
!pip install gym

In [ ]:
import gym

In [ ]:
class TradingAI(gym.Env):
  def __init__(self):
        self.action_space = gym.spaces.Discrete(5)
        self.observation_space = gym.spaces.Box(low=-1,high=1,shape=(118,))

        self.account_balance = 100
        self.trading_fee = 0.01
        self.max_trades = 100000
        self.current_trades = 0
        self.profit = 0

  def load_dataset(self):
        self.dataset = pd.read_parquet("data.parquet")
  
  def choose_and_return(self):
        

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

env = TradingAI()
model = PPO("MlpPolicy", env, verbose=1)


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [ ]:
model.learn(total_timesteps=10_000)